Projection results are saved at /home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/sub-CHUV{sub}/surf/{hemi}.{para}

In [ ]:
import pandas as pd
import nibabel as nib
from nilearn import surface
from nibabel import freesurfer as fs
import os
import numpy as np

# 1. Read the CSV file
csv_path = '/home/localadmin/hpc_mount/wangqiaochu/completed.csv'
df = pd.read_csv(csv_path)

# 2. Define surface types and DKI metrics
surface_types = ['rh.pial', 'rh.white', 'lh.pial', 'lh.white']
dmri_metrics = ['rk', 'ak', 'mk', 'rd', 'fa', 'ad', 'md']

def get_surface_paths(sub):
    base_path = f'/home/localadmin/hpc_mount/wangqiaochu/sub-CHUV{sub}'
    return {
        surf_type: f'{base_path}/{surf_type}'
        for surf_type in surface_types
    }

def get_dmri_paths(sub):
    base_path = f'/home/localadmin/hpc_mount/wangqiaochu/sub-CHUV{sub}'
    return {
        metric: f'{base_path}/{metric}.nii.gz'
        for metric in dmri_metrics
    }

def apply_threshold(data, metric):
    """Apply thresholding based on metric type"""
    if metric in ['rk', 'ak', 'mk']:  # DWI metrics
        mask = (data > 0) & (data < 10)
    elif metric == 'fa': 
        mask = (data > 0) & (data < 1)
    elif metric in ['rd', 'ad', 'md']:
        mask = (data > 0) & (data < 4)
    
    data_thresholded = data.copy()
    data_thresholded[~mask] = np.nan
    return data_thresholded

def process_subject(sub):
    print(f"Processing subject sub-CHUV{sub}")
    surface_paths = get_surface_paths(sub)
    dmri_paths = get_dmri_paths(sub)
    results = {}
    
    for hemi in ['lh', 'rh']:
        pial_surf = surface_paths[f'{hemi}.pial']
        white_surf = surface_paths[f'{hemi}.white']
        
        for metric in dmri_metrics:
            img_path = dmri_paths[metric]
            if not os.path.exists(img_path):
                print(f"Warning: {img_path} does not exist. Skipping.")
                continue
            img = nib.load(img_path)
            data = img.get_fdata()
            
            data_thresholded = apply_threshold(data, metric)   
            img_thresholded = nib.Nifti1Image(data_thresholded, img.affine, img.header)
                    
            # Project the volume data to the surface with the averaged value between white and pial surfaces
            projected_data = surface.vol_to_surf(
                img_thresholded,
                surf_mesh=pial_surf,
                interpolation='linear',
                kind='depth',
                inner_mesh=white_surf,
                depth=None
            )
            # https://nilearn.github.io/stable/modules/generated/nilearn.surface.vol_to_surf.html
            results[f'{hemi}_{metric}'] = projected_data
            
    output_dir = f'/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/sub-CHUV{sub}/surf'
    for key, data in results.items():
        hemi, metric = key.split('_')
        filename = f'{hemi}.{metric}'
        output_path = os.path.join(output_dir, filename)
        fs.write_morph_data(output_path, data)
        print(f"Saved {output_path}")
    
    return results

for _, row in df.iterrows():
    sub = row['sub']
    try:
        subject_results = process_subject(sub)
    except Exception as e:
        print(f"Error processing sub-CHUV{sub}: {str(e)}")


print("Processing complete.")
